# Siamese Training: Training the Model

#### General Steps to Follow

1. Importing Packages
2. Defining x_train, x_test, y_train, y_test
3. Building and training the siamese network
4. Model Evaluation

## 1) Importing Packages

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Input, Flatten, Dense, Conv2D, MaxPooling2D, Layer,Concatenate, Lambda
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy

2023-11-29 15:52:32.313443: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-29 15:52:33.271061: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-29 15:52:33.275769: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-29 15:52:34.647396: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


### ----------------------------------------------------------------------------------------------------------------------------------------------------------

## 2) Defining x_train, x_test, y_train, y_test

#### Loading the training and test data from "other data" folder

In [2]:
train_data = np.load("../other data/train_data.npy", allow_pickle = True)
test_data = np.load("../other data/test_data.npy"  , allow_pickle = True)

* x_train and x_test will contain pairs of the anchor image and the validation image(positive or negative image).
* y_train and y_test will contain the label of each pair:
  - 1 if the pairs are similar images.
  - 0 if the pairs are different images.

In [3]:
x_train = train_data[:,0:2]
y_train = train_data[:,2]
x_test = test_data[:,0:2]
y_test = test_data[:,2]

#### Reshaping the input

In [4]:
x1_train = x_train[:,0]                   #anchor images
x1_train = np.array(x1_train.tolist())    
x2_train = x_train[:,1]                   #validation images(positive/negative)
x2_train = np.array(x2_train.tolist())

x1_test = x_test[:,0]                    #anchor images
x1_test = np.array(x1_test.tolist())
x2_test = x_test[:,1]                    #validation images(positive/negative)
x2_test = np.array(x2_test.tolist())

y_train = tf.convert_to_tensor(y_train.tolist())
y_test = tf.convert_to_tensor(y_test.tolist())

2023-11-29 15:52:51.753365: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


#### Checking the shapes

In [5]:
print("Train Data:")
print("Shape of anchor images    : ", x1_train.shape)
print("Shape of validation images: ", x2_train.shape)
print("Shape of labels           : ", y_train.shape)

print("--------------------------------------------------")

print("Test Data:")
print("Shape of anchor images    : ", x1_test.shape)
print("Shape of validation images: ", x2_test.shape)
print("Shape of labels           : ", y_test.shape)

Train Data:
Shape of anchor images    :  (896, 105, 105, 3)
Shape of validation images:  (896, 105, 105, 3)
Shape of labels           :  (896,)
--------------------------------------------------
Test Data:
Shape of anchor images    :  (224, 105, 105, 3)
Shape of validation images:  (224, 105, 105, 3)
Shape of labels           :  (224,)


### ----------------------------------------------------------------------------------------------------------------------------------------------------------

## 3) Building and training the siamese network

### 3.1 Building the base of the network

In [11]:
inp_shape = [105,105,3]

In [12]:
def make_base_network():
    model = Sequential(
        [
            Input(shape = inp_shape, name = "input_image"),
            Conv2D(64, (10, 10), activation = 'relu'),
            MaxPooling2D(64, (2,2), padding = 'same'),
            
            Conv2D(128, (7, 7), activation = 'relu'),
            MaxPooling2D(64, (2,2), padding = 'same'),
            
            Conv2D(128, (4, 4), activation = 'relu'),
            MaxPooling2D(64, (2,2), padding = 'same'),
            
            Conv2D(256, (4, 4), activation = 'relu'),
            
            Flatten(),
            
            Dense(4096, activation = 'sigmoid')
        ], name = "BaseNetwork"
    )
    
    return model

In [15]:
base_model = make_base_network()

2023-11-29 15:07:41.167420: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 150994944 exceeds 10% of free system memory.
2023-11-29 15:07:41.211708: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 150994944 exceeds 10% of free system memory.


In [16]:
base_model.summary()

Model: "BaseNetwork"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 96, 96, 64)        19264     
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 48, 48, 64)       0         
 2D)                                                             
                                                                 
 conv2d_5 (Conv2D)           (None, 42, 42, 128)       401536    
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 21, 21, 128)      0         
 2D)                                                             
                                                                 
 conv2d_6 (Conv2D)           (None, 18, 18, 128)       262272    
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 9, 9, 128)        

### --------------------------------------------------------------------------------

### 3.2 Building tthe L1Dist layer

In [12]:
class L1Dist(Layer):
    def __init__(self, **kwargs):
        super(L1Dist, self).__init__(**kwargs)

    def call(self, anchor, validation):
        return tf.abs(anchor - validation)

### --------------------------------------------------------------------------------

### 3.3 Defining the siamese model

In [35]:
def make_siamese_model():
    
    # Anchor input image to the network
    anc_image = Input(shape = inp_shape, name = "input_image")
    
    # Validation input image to the network
    validation_image = Input(shape = inp_shape, name = "Validation_image")
    
    # creating a base model
    base_model = make_base_network()
    
    # Encoding the anchor image
    anchor = base_model(anc_image)

    # Encoding the validation image
    validation = base_model(validation_image)
    
    # Using L1Dist Layer to calculate the L1 distance between the two encodings
    distance_layer = L1Dist()
    distance_layer._name = "distance_layer"
    distance = distance_layer(anchor, validation)

    
    # Defining the output layer
    output_layer = Dense(1, activation = 'linear')(distance)
    
    siamese_model = Model(inputs = [anc_image, validation_image], outputs = output_layer, name = "SiameseNetwork")
    
    return siamese_model

In [36]:
siamese_model = make_siamese_model()
siamese_model.summary()

Model: "SiameseNetwork"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_image (InputLayer)       [(None, 105, 105, 3  0           []                               
                                )]                                                                
                                                                                                  
 Validation_image (InputLayer)  [(None, 105, 105, 3  0           []                               
                                )]                                                                
                                                                                                  
 BaseNetwork (Sequential)       (None, 4096)         231635904   ['input_image[0][0]',            
                                                                  'Validation_image[0

### --------------------------------------------------------------------------------

### 3.4 Compiling and training the siamese model

In [43]:
siamese_model.compile(
    optimizer = Adam(learning_rate = 0.0001),
    loss = BinaryCrossentropy(from_logits = True)
)

In [1]:
siamese_model.fit([x1_train, x2_train], y_train, epochs = 10)

### ----------------------------------------------------------------------------------------------------------------------------------------------------------

## 4) Model Evaluation

In [61]:
def model_eval(y, y_hat):
    m = y.shape[0]
    
    y_hat = y_hat.numpy()
    for i in range(len(y_hat)):
        if(y_hat[i] >= 0.5):
            y_hat[i] = 1
        else:
            y_hat[i] = 0
    
    
    accuracy = 100*(np.sum(y == y_hat)/m)
    print("Accuracy =", accuracy)

#### Evaluation on training data

In [67]:
output1 = siamese_model.predict([x1_train, x2_train])

7/7 [==============================] - 6s 809ms/step


In [68]:
y_hat = tf.nn.sigmoid(output1)
y = y_train
model_eval(y, y_hat)

Accuracy = 100.0


#### Evaluation on test data

In [69]:
output2 = siamese_model.predict([x1_test, x2_test])

7/7 [==============================] - 6s 817ms/step


In [70]:
y_hat = tf.nn.sigmoid(output2)
y = y_test
model_eval(y, y_hat)

Accuracy = 98.5


### Saving the model

In [5]:
siamese_model.save("../other data/siamese_model.h5")

### Loading the model

In [4]:
model = tf.keras.models.load_model('../my data/siamese_model.h5', 
                                   custom_objects={'L1Dist':L1Dist, 'BinaryCrossentropy':tf.losses.BinaryCrossentropy})